---
title: "ODE Examples"
subtitle: "BMES 678: Programming Assignment"
author: "Tony Kabilan Okeke"
format: PrettyPDF-pdf
---

## Glycolytic Oscillations [40%]

Consider the Glycolytic Oscillations described in: [Bier et.a., Control analysis of glycolytic oscillations](https://sacan.biomed.drexel.edu/lib/exe/fetch.php?rev=&media=course:sim2:odeexamples:glycolyticoscillations.bier96.pdf), BiophysChem, 1996.

Replicate Fig.3 in that paper by numerically simulating the extended model. Only plot the species shown in Fig.3 of the paper.

Determine the initial concentrations of H,I,T, and G by “eye-ball”ing Fig.3. Use 0.1 for any initial concentration not specified in their paper. Run the simulation up to 250 seconds. Show the concentrations of H,I,T,G between 0 to 250 seconds.

Note that your plot will not look identical to the one shown in the paper. In the paper, “Fig. 3 shows the eventual limit cycle in a simulation of the full model.”, which means they cropped the initial portion of the simulation. Their time axis (x-axis) in Figure3 is therefore mislabeled and does not start from real simulation time=0, but they start their plot at some real simulation time>0.



## Three-node Repressive Network [60%]

Consider the following three-node repressive network described in: Mogilner et.al., ["Review Quantitative Modeling in Cell Biology: What Is It Good for?"](https://sacan.biomed.drexel.edu/lib/exe/fetch.php?rev=&media=course:sim2:odeexamples:quantitativebiology.whatisitgoodfor.pdf), Mogilner, 2006. Implement this network as a system of ODEs and numerically simulate it. The requirement from your solution is that you obtain and demonstrate a similar oscillating behavior.

The parameters are available from an earlier paper, ["A biochemical oscillator explains several aspects of Myxococcus xanthus behavior during development"](https://sacan.biomed.drexel.edu/lib/exe/fetch.php?rev=&media=course:sim2:odeexamples:myxococcus%20xanthus.biochemical%20oscillator.pdf), Igoshin, 2004. Use all of those parameter values as is, except use “kamax” as a constant kamax=0.08 (this is the same parameter as denoted by “kc1” in the Mogilner paper). Use all initial concentrations set to 0.5 (so that e.g. A+Ap = Atotal = 1). They have renamed the parameters between the two papers, you have to identify which parameter maps to which parameter in the two papers.

The paper uses a kamax parameter that varies with time, whereas you are asked to use a constant kamax in this assignment. Because of this difference, your result will not be identical to that of the paper. However, your result should still display an oscillatory behavior in the concentration of the species.

![](hw3noderepressive.png)